We consider two approaches to variable selection, subset selection and shrinkage-based methods. Subset selection methods look among all possible subsets of variables for the one that minimises some suitable estimate of prediction error. The search problem becomes infeasible for large $p$, and non-exhaustive greedy search methods have to be employed.

Shrinkage-based variable selection methods will instead penalise the RSS by a penalty term that forces the LS regression coefficients to shrink in a manner that favours exact zeros in $\hat\beta$.

The best subset method takes as input a training dataset $\mathcal{T}$ and outputs a $p \times p$ matrix $B$, whose $j$-th column contains $\hat\beta^{\mathcal{M}_j}$ for the best performing model (in the sense of RSS) of size $j$, $\mathcal{M}_j$:
\begin{equation}
    \mathcal{M}_j(\mathcal{T}) = \underset{\mathcal{M} : \|M\| = j}{\operatorname{argmin}} RSS(\hat\beta^{\mathcal{M}}(\mathcal{T}); \mathcal{T}).
\end{equation}


In [1]:
import numpy as np
import itertools

def best_subset(X, y):
    '''
    Computes the best subset selection for linear regression.
    Args:
        X: N x p matrix of covariates (standardised/centered).
        y: N-dimensional vector of responses.
    Returns:
        B: p x p matrix. The j-th column (index j-1) contains
           the coefficients of the best model of size j.
    '''
    N, p = X.shape
    B = np.zeros((p, p))

    # Iterate over every possible model size j from 1 to p
    for j in range(1, p + 1):
        best_rss = np.inf
        best_beta_subset = None
        best_indices = None

        # Iterate over all unique combinations of columns of size j
        # combinations returns tuples of indices like (0, 1, 4)
        for indices in itertools.combinations(range(p), j):
            X_subset = X[:, indices]

            # Fit OLS: beta = (X'X)^-1 X'y
            beta_subset, residues, rank, s = np.linalg.lstsq(X_subset, y, rcond=None)

            # Calculate RSS
            y_pred = X_subset @ beta_subset
            rss = np.sum((y - y_pred)**2)

            # Update best model if this subset has lower RSS
            if rss < best_rss:
                best_rss = rss
                best_beta_subset = beta_subset
                best_indices = indices

        # Store the best coefficients in the j-th column of B
        # Map subset coefficients back to their original positions
        if best_indices is not None:
            B[list(best_indices), j-1] = best_beta_subset

    return B

The model space is the set of all possible subsets of the covariates $\{1, \dots, p\}$. The size of the model space is $|\mathcal{M}| = 2^p$ (including the empty set). Typically, we need to find the smallest integer $p$ such that the number of subsets for a specific size $j$, given by the binomial coefficient $\binom{p}{j}$, exceeds a given upper bound $C$,
\begin{equation}
    \binom{p}{j} > C \quad \text{for some } j \in \{1, \dots, p\}.
\end{equation}
The binomial coefficient is maximised when $j$ is in the middle of the range, i.e., $j \approx p/2$. This gives the feasible range for this method.

---

The method greedy subset performs the same proceedure but it incrementally builds up the model sequence $\mathcal{M}_j$ by adding at each iteration the covariate that improves model fit the most
\begin{equation}
    \mathcal{M}_0 = \emptyset, \quad \mathcal{M}_{d+1}(\mathcal{T}) = \mathcal{M}_d(\mathcal{T}) \cup \left\{ l | l = \underset{j}{\operatorname{argmin}} RSS (\hat\beta^{\mathcal{M}_d(\mathcal{T}) \cup \{j\}}(\mathcal{T}); \mathcal{T})\right\}.
\end{equation}

In [2]:
def greedysubset(X, y):
    '''
    Computes the greedy subset selection (forward stepwise) for linear regression.
    Args:
        X: N x p matrix of covariates.
        y N-dimensional vector of responses.
    Returns:
        B: p x p matrix. The j-th column contains
           the coefficients of the greedy model of size j.
    '''
    N, p = X.shape
    B = np.zeros((p, p))

    # Track selected and remaining feature indices
    selected_indices = []
    remaining_indices = list(range(p))

    # Iteratively build models of size 1 to p
    for size in range(1, p + 1):
        best_rss = np.inf
        best_candidate = -1
        best_beta_subset = None

        # Iterate through all remaining candidates to find the best addition
        for candidate in remaining_indices:
            # Trial subset: currently selected + candidate
            trial_indices = selected_indices + [candidate]
            X_subset = X[:, trial_indices]

            # Fit OLS
            beta_subset, _, _, _ = np.linalg.lstsq(X_subset, y, rcond=None)

            # Compute RSS
            y_pred = X_subset @ beta_subset
            rss = np.sum((y - y_pred)**2)

            # Check if this is the best improvement
            if rss < best_rss:
                best_rss = rss
                best_candidate = candidate
                best_beta_subset = beta_subset

        # Permanently add the best candidate to the model
        selected_indices.append(best_candidate)
        remaining_indices.remove(best_candidate)

        # Store coefficients in the output matrix B
        # Map subset coeffs to correct row indices.
        B[selected_indices, size - 1] = best_beta_subset

    return B

The nested property of the models $\mathcal{M}_0 \subset \mathcal{M}_1 \subset \dots \subset \mathcal{M}_p$ can be exploited for computational efficiency. Instead of solving the LS problem from scratch for every step (which involves inverting a matrix or solving a system of equations at a cost of $O(j^3)$), we can update the inverse of the Gram matrix $(X^T X)^{-1}$ from the previous step.

Let $X_j$ be the design matrix for the model with $j$ variables, and let $(X_j^T X_j)^{-1}$ be known. When we move to model $\mathcal{M}_{j+1}$, we append a new column vector $x$ to $X_j$, creating $X_{j+1} = \begin{pmatrix} X_j & x \end{pmatrix}$. The new Gram matrix is a block matrix:
\begin{equation}
    X_{j+1}^T X_{j+1} =
    \begin{pmatrix}
        X_j^T X_j & X_j^T x \\
        x^T X_j & x^T x
    \end{pmatrix} =
    \begin{pmatrix}
        A & b \\
        b^T & c
    \end{pmatrix}
\end{equation}
where $A = X_j^T X_j$. Using the block matrix inversion formula, the inverse is
\begin{equation}
    (X_{j+1}^T X_{j+1})^{-1} =
    \begin{pmatrix}
        A^{-1} + \frac{1}{k} (A^{-1}b)(A^{-1}b)^T & -\frac{1}{k} A^{-1}b \\
        -\frac{1}{k} b^T A^{-1} & \frac{1}{k}
    \end{pmatrix}
\end{equation}
where $k = c - b^T A^{-1} b$ is the Schur complement. The upper left $j \times j$ block of the new inverse is $A^{-1} + \frac{1}{k} (A^{-1}b)(A^{-1}b)^T$ which can be computed from the old inverse by adding a rank-$1$ update matrix. This update only relies on matrix-vector multiplications which only costs $O(j^2)$.

---

We can amend the greedy subset method to stop whenever the newly added variable does not significantly improve fit, using the $F$-statistic
\begin{equation}
    \frac{RSS(\hat\beta^{\mathcal{M}_d}) - RSS(\hat\beta^{\mathcal{M}_{d+1}})}{RSS(\hat\beta^{\mathcal{M}_{d+1}})/(N - d - 1)},
\end{equation}
which follows an $F_{1,N-d-1}$ distribution.

In [3]:
from scipy.stats import f

def forward_f_test_subset(X, y, alpha=0.05):
    '''
    Performs forward stepwise selection with an F-test stopping criterion.
    Args:
        X: N x p matrix of covariates.
        y: N-dimensional vector of responses.
        alpha: Significance level.
    Returns:
        model_indices: List of indices of the selected covariates.
    '''
    N, p = X.shape
    selected_indices = []
    remaining_indices = list(range(p))

    # Calculate RSS for the initial empty model (M_0)
    # Since variables are zero-mean, the intercept-only prediction is 0.
    # RSS_0 = sum((y - 0)^2) / N = mean(y^2)
    current_rss = np.mean(y**2)

    # We iterate to potentially add p variables
    for d in range(p):
        best_rss = np.inf
        best_candidate = -1
        best_beta = None

        # Greedy Step: Find the best variable to add
        for candidate in remaining_indices:
            trial_indices = selected_indices + [candidate]
            X_subset = X[:, trial_indices]

            # Fit OLS
            beta_subset, _, _, _ = np.linalg.lstsq(X_subset, y, rcond=None)

            # Compute RSS (Mean Squared Error formulation)
            y_pred = X_subset @ beta_subset
            rss_trial = np.mean((y - y_pred)**2)

            if rss_trial < best_rss:
                best_rss = rss_trial
                best_candidate = candidate

        # F-Test Step: Compare M_d with M_{d+1}
        # F = (RSS_d - RSS_{d+1}) / (RSS_{d+1} / (N - d - 1))

        numerator = current_rss - best_rss
        denominator = best_rss / (N - d - 1)

        # Avoid division by zero if perfect fit
        if denominator == 0:
            F_stat = np.inf
        else:
            F_stat = numerator / denominator

        # Compute p-value using F distribution with df1=1, df2=N-d-1
        p_value = 1 - f.cdf(F_stat, 1, N - d - 1)

        # Stopping Criterion
        if p_value > alpha:
            print(f"Stopping at size {d}. Best candidate (idx {best_candidate}) "
                  f"p-value: {p_value:.4f} > {alpha}")
            break

        # If significant, update model and continue
        selected_indices.append(best_candidate)
        remaining_indices.remove(best_candidate)
        current_rss = best_rss

    return selected_indices

This method would not work for best subset selection since the $F$-test statistic is designed for nested hypothesis testing. It measures the statistical significance of the reduction in error variance when adding parameters to an existing model.

Furthermore, even if the models happened to be nested, applying an $F$-test to the best subset found introduces a selection bias. The $F$-statistic assumes that the model structure was chosen a priori. When we select the variable that maximises the reduction in RSS from a set of $\binom{p}{d}$ combinations, the resulting $F$-statistic will be inflated. The distribution of this maximum statistic is not the standard $F_{1, N-d-1}$, meaning the $p$-values would be too small, leading to the inclusion of false positives.

---

We can represent a sparse (linear regression) estimator more generally as
an algorithm that takes as input a training set $\mathcal{T}$ and outputs a sequence of $p$ candidate regression vectors for each model size (i.e., the $j$-th candidate $\hat\beta(j)(\mathcal{T})$ has precisely $p - j$ zeros). Best and greedy subset search are special cases of this definition for which each
candidate is a least squares solution, a condition we will not insist on here.

We would like to select among candidates on the basis of estimated prediction error $\hat{PE}$:
\begin{equation}
    \hat\beta^{CV}(\mathcal{T}) = \hat\beta^{j^*}(\mathcal{T}), \quad \text{where} j^* = \underset{j}{\operatorname{argmin}} \{\hat{PE}(j, \mathcal{T})\}.
\end{equation}
The prediction error can be estimated using $10$-fold cross-validation as
\begin{equation}
    \hat{PE}(j, \mathcal{T}) = \frac{1}{10}\sum_{k=1}^{10} RSS(\hat\beta^{(j)}(\mathcal{T}^{-k}); \mathcal{T}^k),
\end{equation}
where $\mathcal{T}^k$ is the $k$-th fold of the training set and $\mathcal{T}^{-k}$ its complement
\begin{align}
    \mathcal{T}^k &= \left\{ (y_{\pi(n)}, x_{\pi(n)}) | k - 1 < \frac{10n}{N} \leq k \right\}, \\
    \mathcal{T}^{-k} &= \left\{(y_{\pi(n)}, x_{\pi(n)}) | \frac{10n}{N} \leq k − 1 \text{ or } \frac{10n}{N} > k \right\},
\end{align}
where $\pi$ is a random permutation of $\{1, \dots, N\}$.


In [4]:
def crossval(X, y, sparse_estimator):
    '''
    Performs 10-fold cross-validation to select the best model size.
    Args:
        X: N x p matrix of covariates.
        y: N-dimensional vector of responses.
        sparse_estimator: A function that takes (X, y) and returns a p x p matrix
                          of coefficients (columns correspond to model sizes 1 to p).
    Returns:
        beta_cv: The coefficient vector of the best performing model.
    '''
    N, p = X.shape
    K = 10

    # Create random permutation of indices
    indices = np.random.permutation(N)

    # Split indices into 10 roughly equal folds
    folds = np.array_split(indices, K)

    # Matrix to store RSS for each fold (rows) and each model size (cols)
    rss_matrix = np.zeros((K, p))

    # Iterate through each fold
    for k in range(K):
        # The k-th fold is the validation set
        test_idx = folds[k]

        # The complement is the training set
        mask = np.ones(N, dtype=bool)
        mask[test_idx] = False
        train_idx = np.where(mask)[0]

        X_tr, y_tr = X[train_idx], y[train_idx]
        X_te, y_te = X[test_idx], y[test_idx]

        # Compute candidate models on training Set
        # B_matrix is p x p, where col j corresponds to model size j+1
        B_matrix = sparse_estimator(X_tr, y_tr)

        # Evaluate prediction error on validation set for all sizes
        for j in range(p):
            # Coefficients for model size j+1
            beta_j = B_matrix[:, j]
            # Prediction
            y_pred = X_te @ beta_j
            # Compute RSS
            rss_matrix[k, j] = np.sum((y_te - y_pred)**2)

    # Select best model size
    # Calculate mean RSS across the 10 folds for each model size
    mean_rss = np.mean(rss_matrix, axis=0)

    # Find the index minimising the error (j*)
    best_j_index = np.argmin(mean_rss)

    # Refit on full dataset
    # Return the estimator trained on complete dataset T selected at optimal size j*.
    B_final = sparse_estimator(X, y)
    beta_cv = B_final[:, best_j_index]

    return beta_cv